## Description 
This example demonstrates how to run GeniERuntime through OneWorkflow. A small dummy script for demonstrating SifIO is also provided. This example uses a parametrized containership model from [GeniE Snack Pack](https://sesam.dnv.com/genie_utils/hullforms/containership.html). *AP* and *FP* are parametrized and input given from a Pandas dataframe as shown below.




In [ ]:
from dnv.oneworkflow.utils import *
from pathlib import Path
import os
root_folder = os.getcwd()
workspacePath = str(Path(root_folder, 'Workspace'))
workspaceId = "GeniERuntimeExample"

cloudRun = False
oneWorkflowTMPFolder = r'c:\oneworkflowTmp' #due to possible issues with long file paths we prefer to have this folder at the root
if not os.path.exists(oneWorkflowTMPFolder):
    try:
        print("Trying to create tmp folder for one workflow local execution")
        os.mkdir(oneWorkflowTMPFolder)
        print(oneWorkflowTMPFolder + " created!\n")
    except:
        print("did not manage to create tmp folder for local execution. Check that you have privileges to create it or try to manually create it from the coomand line.")
#If running locally the code below will also start the local workflow host.
workflow_client = one_workflow_client(workspace_id = workspaceId, cloud_run = cloudRun, workspace_path = workspacePath, local_workflow_runtime_temp_folder_path = oneWorkflowTMPFolder,
                                      local_workflow_runtime_temp_folders_cleanup=False,environment=Environment.Testing,pool_id="SesamWorkflow_Windows_Standard_A2_v2")


In [ ]:
import pandas as pd
import shutil
from dnv.oneworkflow import PythonCommand
from dnv.sesam.commands import *
# we must delete existing results locally before generating new results
local__result_path = Path(workspacePath, workflow_client.results_directory)
try:
    if os.path.isdir(local__result_path):
        shutil.rmtree(local__result_path)
except:
      print("Error while deleting the result folder")

#parametrized values
df = pd.DataFrame({'AP':  ["0m", "0.5m", "1m"], 'FP': ["150m", "250m", "500m"]})
#for debugging only
#job_json = json.dumps(job, default=lambda o: o.encode(), indent=4)
#print(job_json)


commands_info = []
for index, row in df.iterrows():
            loadcase_folder_name = f"Model_{index + 1}"
            genieruntime_command = GeniERuntimeCommand()
            genieruntime_command.Parameters = {}
            genieruntime_command.TemplateInputFile = "ContainerHull_template.js"
            for key, value in row.items():
                genieruntime_command.Parameters[key] = value
           
            post_processing_command = PythonCommand(
                directory=workflow_client.common_directory,
                filename="postprocessing.py")
            cmd_info = CommandInfo(commands=[genieruntime_command,post_processing_command],load_case_foldername=loadcase_folder_name)
            commands_info.append(cmd_info)
            
print("Running commands in parallel")
await run_managed_commands_parallelly_async(
            client=workflow_client,
            commands_info=commands_info,
            files_to_download_from_blob_to_client=FileTransferOptions(max_size="11124MB",patterns=["**/*.txt", "**/*.lis", "**/*.MLG", "**/*.MLG","**/*.CSV","**/*.FEM"]),
            enable_common_files_copy_to_load_cases=True,
)


## Postprocessing
The code below prints out node and element counts for the different models and present the results in a table.


In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import glob

lc = 11
frames = []
for folder in glob.glob(f"{local__result_path}\\*"):
    data = pd.read_csv(Path(folder, "nodeCount.csv"), index_col=0)
    data.index = [folder.split('\\')[-1]]
    frames.append(data)
df = pd.concat(frames)
df=df.rename_axis("Loadcase")
display(df)
